Amazon US Customer Reviews - Market Basket Analaysis

In [ ]:
from google.colab import files
import os
import json
import getpass

# Prompt the user for input
use_upload = input('Do you want to upload the Kaggle credentials file? (y/n): ')
if use_upload.lower() == 'y':
    # Upload the file
    uploaded = files.upload()

    # Rename the file to KaggleCredential.json
    for name in uploaded.keys():
        if 'kaggle' in name.lower():
            os.rename(name, 'KaggleCredential.json')
            break
else:
    # Manually input the credentials
    username = input('Enter your Kaggle username: ')
    password = getpass.getpass('Enter your Kaggle key: ')

    # Save the credentials to a JSON file
    credentials = {'username': username, 'key': password}
    with open('KaggleCredential.json', 'w') as f:
        json.dump(credentials, f)

# Load the credentials from the JSON file
with open('KaggleCredential.json', 'r') as f:
    credentials = json.load(f)

# Get the Kaggle username from the credentials
username = credentials['username']
password = credentials['key']
print(username)

Do you want to upload the Kaggle credentials file? (y/n): n
Enter your Kaggle username: lucapaoletti
Enter your Kaggle key: ··········
lucapaoletti


In [ ]:
import getpass
import json

# Save your credentials to a JSON file
credentials = {'username': username, 'key': password}
kaggle_dir = '/root/.kaggle'
if not os.path.exists(kaggle_dir):
    os.makedirs(kaggle_dir)
with open(os.path.join(kaggle_dir, 'kaggle.json'), 'w') as f:
    json.dump(credentials, f)

# Set the file permissions to read/write only for the owner
!chmod 600 /root/.kaggle/kaggle.json

In [ ]:
import kaggle
datasets = !kaggle datasets list -s 'Amazon US Customer Reviews'
datasets

['ref                                                 title                               size  lastUpdated          downloadCount  voteCount  usabilityRating  ',
 '--------------------------------------------------  ----------------------------------  ----  -------------------  -------------  ---------  ---------------  ',
 'cynthiarempel/amazon-us-customer-reviews-dataset    Amazon US Customer Reviews Dataset  21GB  2021-06-16 20:07:46           3182         31  1.0              ',
 'mahran34/amazon-customer-reviews-for-some-products  Amazon reviews                       1MB  2021-10-23 17:49:34             50          1  0.47058824       ',
 'washingtongold/amazon-us-software-reviews           Amazon US Software Reviews          99MB  2022-02-20 16:56:08             36          1  0.5882353        ']

In [ ]:
kaggle.api.dataset_list_files('cynthiarempel/amazon-us-customer-reviews-dataset').files #list of file in main dataset

[amazon_reviews_us_Grocery_v1_00.tsv,
 amazon_reviews_us_Mobile_Electronics_v1_00.tsv,
 amazon_reviews_us_Video_DVD_v1_00.tsv,
 amazon_reviews_us_Outdoors_v1_00.tsv,
 amazon_reviews_us_Software_v1_00.tsv,
 amazon_reviews_us_Digital_Video_Download_v1_00.tsv,
 amazon_reviews_us_Mobile_Apps_v1_00.tsv,
 amazon_reviews_multilingual_US_v1_00.tsv,
 amazon_reviews_us_Digital_Ebook_Purchase_v1_01.tsv,
 amazon_reviews_us_Digital_Software_v1_00.tsv,
 amazon_reviews_us_Electronics_v1_00.tsv,
 amazon_reviews_us_Toys_v1_00.tsv,
 amazon_reviews_us_Office_Products_v1_00.tsv,
 amazon_reviews_us_Shoes_v1_00.tsv,
 amazon_reviews_us_Beauty_v1_00.tsv,
 amazon_reviews_us_Musical_Instruments_v1_00.tsv,
 amazon_reviews_us_Automotive_v1_00.tsv,
 amazon_reviews_us_Camera_v1_00.tsv,
 amazon_reviews_us_Furniture_v1_00.tsv,
 amazon_reviews_us_Apparel_v1_00.tsv,
 amazon_reviews_us_Books_v1_02.tsv,
 amazon_reviews_us_Baby_v1_00.tsv,
 amazon_reviews_us_Tools_v1_00.tsv,
 amazon_reviews_us_PC_v1_00.tsv,
 amazon_reviews

In [ ]:
# Import necessary libraries
import kaggle
import zipfile

# Authenticate Kaggle API
kaggle.api.authenticate()

In [ ]:
kaggle.api.dataset_download_file('cynthiarempel/amazon-us-customer-reviews-dataset','amazon_reviews_us_Digital_Software_v1_00.tsv') # choose dataset among the ones above

True

In [ ]:
# Extract the zip file to the current working directory (i.e., the root directory in Colab)
with zipfile.ZipFile('amazon_reviews_us_Digital_Software_v1_00.tsv.zip', 'r') as zip_ref:
    zip_ref.extractall()  # Extract to the current directory

In [ ]:

from pyspark import SparkConf, SparkContext
from pyspark.sql import SparkSession
from pyspark.sql.functions import collect_set, col, count
from pyspark.sql import SQLContext
     


In [ ]:
conf = SparkConf().setAppName("MarketBasket")
conf = conf.setMaster('local[*]')
sc = SparkContext(conf=conf)

Dataset import as RDD

In [ ]:
sqlContext = SQLContext(sc)
# import dataset
df = sqlContext.read.csv('amazon_reviews_us_Digital_Software_v1_00.tsv', header=True, sep = '\t') 
# subset dataset
df = df.select(col("customer_id"),col("product_id"))

In [ ]:
# sum function
def sum_prod(x,y):
    return x+y

# subset or not function
def filtering(rddlist, filt):
  for item in filt:
    if set(list(item)).issubset(set(rddlist)):
      return ((item, 1))

# A-priori algorithm

In [ ]:
def apriori(rdd, threshold):
  
  flat_list  = rdd.flatMap(list) 

  singleton = flat_list.map(lambda item: (item , 1)) # (item, 1)
  singleton_summed = singleton.reduceByKey(sum_prod) # sum by key
  singleton_filtered = singleton_summed.filter(lambda item: item[1] >= threshold ) # check threshold

  # items filtered
  frequent_prod = singleton_filtered.map(lambda item: (item[0]))

  # candidate pairs
  pairs_list = list(itertools.combinations(frequent_prod.toLocalIterator(),2))

  # pairs are subset of baskets?
  support_table_pairs = rdd.map(lambda x : filtering(x, pairs_list)).filter(lambda x: x is not None).cache() 
  support_table_pairs_summed = support_table_pairs.reduceByKey(sum_prod) # sum by key
  support_table_pairs_filtered = support_table_pairs_summed.filter(lambda item: item[1] >= threshold) # check threshold

  return (support_table_pairs_filtered)


# SON algorithm 

In [ ]:
# available baskets
baskets = df.groupBy("customer_id").agg(collect_set("product_id").alias("product_id"))

# available baskets
basket_list = baskets.select('product_id').rdd.flatMap(list)

# Parallelize the list in RDD
basket_list = sc.parallelize(basket_list.collect(), 5)

In [ ]:
minSupport = 35 # definition of threshold
numPartitions = basket_list.getNumPartitions() # get number of partition (5 in this case)
adjSupport = minSupport/numPartitions # threshold for each partition
adjSupport

In [ ]:
import itertools

In [ ]:
candidates = sc.parallelize([]) # create a RDD to merge the result of apriori applied on each chunk

for i in range(0, numPartitions-1):
  partition = sc.parallelize(basket_list.glom().collect()[i]) # collect each partition
  support_table_pairs_filtered = apriori(partition, adjSupport) # apply apriori to each partition
  candidate_chunk = support_table_pairs_filtered.map(lambda item: (item[0],1)) # (item, 1)
  candidates = candidates.union(candidate_chunk) # all in one

In [ ]:
candidates_prods = candidates.map(lambda item : item[0]) 
candidates_list = candidates_prods.collect() # more time comsuming

In [ ]:
candidates_check = basket_list.map(lambda x : filtering(x, candidates_list)).filter(lambda x: x is not None).cache() # subset or not
candidates_summed = candidates_check.reduceByKey(sum_prod) # sum by key
candidates_filtered = candidates_summed.filter(lambda item: item[1] >= minSupport) # check threshold

# Results

In [ ]:
# list of all products    
flat_list  = basket_list.flatMap(list)

# obtain the products that appear in a number of baskets larger than the threshold:
singleton = flat_list.map(lambda item: (item , 1)) # add one for each product appearence
singleton_summed = singleton.reduceByKey(sum_prod) # sum of values by product as key
singleton_filtered = singleton_summed.filter(lambda item: item[1] >= minSupport ) # consider just the product that performed in more than support value

In [ ]:
singleton_filtered.sortBy(lambda x: x[1], ascending=False).take(10)

[('B00H9A60O4', 9468),
 ('B00NG7JVSQ', 6308),
 ('B00E7X9RUK', 2245),
 ('B00M9GTHS4', 2208),
 ('B00PG8FOSY', 1936),
 ('B008S0IMCC', 1923),
 ('B00FGDDTSQ', 1800),
 ('B00M76N6MO', 1686),
 ('B0064PFB9U', 1585),
 ('B00B1TGUMG', 1450)]

In [ ]:
candidates_filtered.sortBy(lambda x: x[1], ascending=False).take(10)

[(('B008SCNLEY', 'B00FGDDTSQ'), 84),
 (('B009HBCU9W', 'B00FFINOWS'), 83),
 (('B00FFINOWS', 'B00NG7JVSQ'), 78),
 (('B00G0DXA9Y', 'B00PG8FOSY'), 63),
 (('B008SCMUUA', 'B00FGDEPDY'), 60),
 (('B00A42LWHO', 'B00G0DXA9Y'), 49),
 (('B00M9GTHS4', 'B00E7X9RUK'), 49),
 (('B00NG7JVSQ', 'B00PG8FOSY'), 43),
 (('B00NG7JVSQ', 'B00NG7K2RA'), 37)]